In [1]:
from custom_dataset.segdataset import InstanceSegmentationDataset
from torch.utils.data import DataLoader
import json

In [2]:
from transformers import SegformerImageProcessor

root_dir = '/disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe'
#----------------------
# reduce_labels should be False !! 

image_processor = SegformerImageProcessor(reduce_labels=False)

train_dataset = InstanceSegmentationDataset(root_dir=root_dir, image_processor=image_processor)
valid_dataset = InstanceSegmentationDataset(root_dir=root_dir, image_processor=image_processor, train=False)

/home/eric/anaconda3/envs/trex/lib/python3.9/site-packages/transformers/models/segformer/image_processing_segformer.py:102: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [3]:
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=2)

In [4]:
json_file = '/disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/_annotations.coco.json'
with open(json_file, 'r') as f:
    data = json.load(f)
#--- 
id2label = {
    0:data['categories'][0]['name'],
    1:data['categories'][1]['name'],
    2:data['categories'][2]['name'],
    3:data['categories'][3]['name'],
    4:data['categories'][4]['name']
}
label2id = {v: k for k, v in id2label.items()}

In [5]:
# define model
from transformers import SegformerForSemanticSegmentation
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/mit-b5",
                                                         num_labels=5,
                                                         id2label=id2label,
                                                         label2id=label2id,
)

/home/eric/anaconda3/envs/trex/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/eric/anaconda3/envs/trex/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b5 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.runnin

In [6]:
model

SegformerForSemanticSegmentation(
  (segformer): SegformerModel(
    (encoder): SegformerEncoder(
      (patch_embeddings): ModuleList(
        (0): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
          (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        )
        (1): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        )
        (2): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(128, 320, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
        )
        (3): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(320, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)

In [7]:
import evaluate

In [ ]:
metric = evaluate.load("mean_iou")

In [9]:
image_processor.do_reduce_labels

False

In [10]:
#-----------
_EPOCHS = 100

#----------

In [20]:
import torch
from torch import nn
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm
#from tqdm import tqdm

# define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00006)
# move model to GPU
device = torch.device("cuda:0")
model.to(device)
model.train()

for epoch in range(_EPOCHS):  # loop over the dataset multiple times
    print("Epoch:", epoch)

    for idx, batch in enumerate(tqdm(train_dataloader)):
          # get the inputs;
          pixel_values = batch["pixel_values"].to(device)
          labels = batch["labels"].to(device)

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = model(pixel_values=pixel_values, labels=labels)
          loss, logits = outputs.loss, outputs.logits

          loss.backward()
          optimizer.step()

          # evaluate
          with torch.no_grad():
            upsampled_logits = nn.functional.interpolate(logits, size=labels.shape[-2:], mode="bilinear", align_corners=False)
            predicted = upsampled_logits.argmax(dim=1)

            # note that the metric expects predictions + labels as numpy arrays
            # metric.add_batch(predictions=predicted.detach().cpu().numpy(), references=labels.detach().cpu().numpy())

          # let's print loss and metrics every 100 batches
        #print(idx)
          if idx % 100 == 0:
            #print(idx)
            # currently using _compute instead of compute
            # see this issue for more info: https://github.com/huggingface/evaluate/pull/328#issuecomment-1286866576
            metrics = metric._compute(
                    predictions=predicted.cpu(),
                    references=labels.cpu(),
                    num_labels=len(id2label),
                    ignore_index=255,
                    reduce_labels=False, # we've already reduced the labels ourselves
                )
            print("Epoch:", epoch)
            print("Loss:", loss.item())
            print("Mean_iou:", metrics["mean_iou"])
            print("Mean accuracy:", metrics["mean_accuracy"])
            #-- save test
            score_ = metrics["mean_iou"]
            #torch.save(model.state_dict(), os.path.join("/data/checkpoints",f"segformer_{epoch}_miou_{score_}_v1") )
        

Epoch: 0


  0%|          | 0/41 [00:00<?, ?it/s]

filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/images/Zamak__part_10.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/labels/Zamak__part_10.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/images/M2A1Slammer2__part_1.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/labels/M2A1Slammer2__part_1.png


/home/eric/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label


Epoch: 0
Loss: 1.686711311340332
Mean_iou: 0.03256391150654143
Mean accuracy: 0.18716518783028613
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/images/Zamak__part_8.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/labels/Zamak__part_8.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/images/M5Sandstorm__part_8.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/labels/M5Sandstorm__part_8.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/images/T140Angara__part_22.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/labels/T140Angara__part_22.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/images/T140Angara__part_17.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/labels/T140Angara__part_17.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/

...

  0%|          | 0/41 [00:00<?, ?it/s]

filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/images/M2A1Slammer2__part_10.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/labels/M2A1Slammer2__part_10.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/images/M2A1Slammer2__part_12.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/labels/M2A1Slammer2__part_12.png
Epoch: 14
Loss: 0.012666735798120499
Mean_iou: 0.914250967124765
Mean accuracy: 0.923036352164216
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/images/T140Angara__part_22.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/labels/T140Angara__part_22.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/images/M5Sandstorm__part_1.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/labels/M5Sandstorm__part_1.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segm

  0%|          | 0/41 [00:00<?, ?it/s]

filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/images/T140Angara__part_24.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/labels/T140Angara__part_24.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/images/M5Sandstorm__part_24.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/labels/M5Sandstorm__part_24.png
Epoch: 15
Loss: 0.009528998285531998
Mean_iou: 0.8973624004599494
Mean accuracy: 0.9795485647326477
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/images/T140Angara__part_25.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/labels/T140Angara__part_25.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/images/T140Angara__part_15.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/labels/T140Angara__part_15.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmenta

KeyboardInterrupt: 

In [ ]:
import os

_EPOCHS = 50
_MODEL_SAVE = False

# Validation Loop
#model.eval()  # Set the model to evaluation mode

for epoch in range(_EPOCHS):  # loop over the dataset multiple times
    print("Epoch:", epoch)

    # Training loop
    for idx, batch in enumerate(tqdm(train_dataloader)):
        # get the inputs
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(pixel_values=pixel_values, labels=labels)
        loss, logits = outputs.loss, outputs.logits

        loss.backward()
        optimizer.step()

        # Evaluate periodically
        if idx % 100 == 0:
            with torch.no_grad():
                upsampled_logits = nn.functional.interpolate(
                    logits, size=labels.shape[-2:], mode="bilinear", align_corners=False
                )
                predicted = upsampled_logits.argmax(dim=1)

                metrics = metric._compute(
                    predictions=predicted.cpu(),
                    references=labels.cpu(),
                    num_labels=len(id2label),
                    ignore_index=255,
                    reduce_labels=False,  # we've already reduced the labels ourselves
                )
                print("Epoch:", epoch)
                print("Loss:", loss.item())
                print("Mean_iou:", metrics["mean_iou"])
                print("Mean accuracy:", metrics["mean_accuracy"])

    # Validation loop
    val_loss = 0
    val_metrics = {"mean_iou": 0, "mean_accuracy": 0}
    for batch in tqdm(valid_dataloader):
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        with torch.no_grad():
            outputs = model(pixel_values=pixel_values, labels=labels)
            val_loss += outputs.loss.item()

            logits = outputs.logits
            upsampled_logits = nn.functional.interpolate(
                logits, size=labels.shape[-2:], mode="bilinear", align_corners=False
            )
            predicted = upsampled_logits.argmax(dim=1)

            batch_metrics = metric._compute(
                predictions=predicted.cpu(),
                references=labels.cpu(),
                num_labels=len(id2label),
                ignore_index=255,
                reduce_labels=False,
            )
            val_metrics["mean_iou"] += batch_metrics["mean_iou"]
            val_metrics["mean_accuracy"] += batch_metrics["mean_accuracy"]

    # Average validation loss and metrics
    val_loss /= len(valid_dataloader)
    val_metrics["mean_iou"] /= len(valid_dataloader)
    val_metrics["mean_accuracy"] /= len(valid_dataloader)

    print(f"Validation Loss: {val_loss}")
    print(f"Validation Mean IoU: {val_metrics['mean_iou']}")
    print(f"Validation Mean Accuracy: {val_metrics['mean_accuracy']}")

    # Save the model if the validation IoU improves
    if val_metrics["mean_iou"] > best_val_iou:
        best_val_iou = val_metrics["mean_iou"]
        
        if _MODEL_SAVE:
            torch.save(
                model.state_dict(),
                os.path.join(
                    "/data/checkpoints",
                    f"segformer_best_epoch_{epoch}_miou_{best_val_iou:.4f}.pt",
                ),
            )
        print("Model saved!")
